# BUSINESS PROBLEM & UNDERSTANDING
STAKEHOLDER:SPOTIFY
PROBLEM: Spotify is launch a new Movie Divison. We are contracted Data Scientist tasked with  creating a reccomendation system . We want to compete with other streaming platforms such as Netflix, Hulu, Amazon Prime. Since they are new to the market, they want to ensure that their Reccomendation System is accurate and reliable.   

 Your task is to:

Build a model that provides top 5 movie recommendations to a user, based on their ratings of other movies.
The MovieLens dataset is a "classic" recommendation system dataset, that is used in numerous academic papers and machine learning proofs-of-concept. You will need to create the specific details about how the user will provide their ratings of other movies, in addition to formulating a more specific business problem within the general context of "recommending movies".




# Data Preparation

In [1]:
#Importing Libraries 
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# Reading in datasets
df_movies = pd.read_csv('data/movies.csv')
df_links = pd.read_csv('data/links.csv')
df_ratings = pd.read_csv('data/ratings.csv')
df_tags = pd.read_csv('data/tags.csv')

In [3]:
#looking at each indivdual csv before merging.
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [4]:
df_links.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [5]:
df_ratings.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


Convertig Ratings and Tags Timestamps to Datetime format

In [7]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'])
df_tags['timestamp'] = pd.to_datetime(df_tags['timestamp'])

In [8]:
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,1970-01-01 00:00:00.964982703
1,1,3,4.0,1970-01-01 00:00:00.964981247
2,1,6,4.0,1970-01-01 00:00:00.964982224
3,1,47,5.0,1970-01-01 00:00:00.964983815
4,1,50,5.0,1970-01-01 00:00:00.964982931
...,...,...,...,...
100831,610,166534,4.0,1970-01-01 00:00:01.493848402
100832,610,168248,5.0,1970-01-01 00:00:01.493850091
100833,610,168250,5.0,1970-01-01 00:00:01.494273047
100834,610,168252,5.0,1970-01-01 00:00:01.493846352


In [9]:
df_tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1970-01-01 00:00:01.445714994
1,2,60756,Highly quotable,1970-01-01 00:00:01.445714996
2,2,60756,will ferrell,1970-01-01 00:00:01.445714992
3,2,89774,Boxing story,1970-01-01 00:00:01.445715207
4,2,89774,MMA,1970-01-01 00:00:01.445715200
...,...,...,...,...
3678,606,7382,for katie,1970-01-01 00:00:01.171234019
3679,606,7936,austere,1970-01-01 00:00:01.173392334
3680,610,3265,gun fu,1970-01-01 00:00:01.493843984
3681,610,3265,heroic bloodshed,1970-01-01 00:00:01.493843978


Merging Dataset to one Pandas Database

In [10]:
# combining the Movies, Ratings, Tags Dataset. Not including Links dataset due to its non relevance to the business problem.
movie_combo = pd.merge(df_movies, df_tags, left_on='movieId', right_on='movieId', how='left')

movie_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11853 entries, 0 to 11852
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   movieId    11853 non-null  int64         
 1   title      11853 non-null  object        
 2   genres     11853 non-null  object        
 3   userId     3683 non-null   float64       
 4   tag        3683 non-null   object        
 5   timestamp  3683 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 648.2+ KB


In [11]:
#adding ratings column to dataset
new_df = movie_combo.merge(df_ratings, how='inner', on = 'movieId')
new_df.drop_duplicates(inplace=True)
new_df

,movieId,title,genres,userId_x,tag,timestamp_x,userId_y,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,1,4.0,1970-01-01 00:00:00.964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,5,4.0,1970-01-01 00:00:00.847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,7,4.5,1970-01-01 00:00:01.106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,15,2.5,1970-01-01 00:00:01.510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,17,4.5,1970-01-01 00:00:01.305696483
...,...,...,...,...,...,...,...,...,...
285757,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN,NaN,NaT,184,4.0,1970-01-01 00:00:01.537109082
285758,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN,NaN,NaT,184,3.5,1970-01-01 00:00:01.537109545
285759,193585,Flint (2017),Drama,NaN,NaN,NaT,184,3.5,1970-01-01 00:00:01.537109805
285760,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN,NaN,NaT,184,3.5,1970-01-01 00:00:01.537110021


In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285762 entries, 0 to 285761
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   movieId      285762 non-null  int64         
 1   title        285762 non-null  object        
 2   genres       285762 non-null  object        
 3   userId_x     233213 non-null  float64       
 4   tag          233213 non-null  object        
 5   timestamp_x  233213 non-null  datetime64[ns]
 6   userId_y     285762 non-null  int64         
 7   rating       285762 non-null  float64       
 8   timestamp_y  285762 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 21.8+ MB


In [13]:
new_df.head()

,movieId,title,genres,userId_x,tag,timestamp_x,userId_y,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,1,4.0,1970-01-01 00:00:00.964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,5,4.0,1970-01-01 00:00:00.847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,7,4.5,1970-01-01 00:00:01.106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,15,2.5,1970-01-01 00:00:01.510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,17,4.5,1970-01-01 00:00:01.305696483


In [14]:
#dropping duplicate Movie entries
new_df.drop_duplicates('movieId', inplace=True)

In [15]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724 entries, 0 to 285761
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   movieId      9724 non-null   int64         
 1   title        9724 non-null   object        
 2   genres       9724 non-null   object        
 3   userId_x     1554 non-null   float64       
 4   tag          1554 non-null   object        
 5   timestamp_x  1554 non-null   datetime64[ns]
 6   userId_y     9724 non-null   int64         
 7   rating       9724 non-null   float64       
 8   timestamp_y  9724 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 759.7+ KB


In [16]:
new_df.head()

,movieId,title,genres,userId_x,tag,timestamp_x,userId_y,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1970-01-01 00:00:01.139045764,1,4.0,1970-01-01 00:00:00.964982703
645,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1970-01-01 00:00:01.528843929,6,4.0,1970-01-01 00:00:00.845553522
1085,3,Grumpier Old Men (1995),Comedy|Romance,289.0,moldy,1970-01-01 00:00:01.143424860,1,4.0,1970-01-01 00:00:00.964981247
1189,4,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN,NaN,NaT,6,3.0,1970-01-01 00:00:00.845554349
1196,5,Father of the Bride Part II (1995),Comedy,474.0,pregnancy,1970-01-01 00:00:01.137373903,6,5.0,1970-01-01 00:00:00.845553938


One Hot Encoding since Genre is a Categorical Column. 

In [17]:
categorical_vars = ["genres"]
one_hot_encoder = OneHotEncoder(sparse=False, drop = "first")

In [18]:
encoder_vars_array = one_hot_encoder.fit_transform(new_df[categorical_vars])
encoder_feature_names = one_hot_encoder.get_feature_names(categorical_vars)
encoder_vars_df = pd.DataFrame(encoder_vars_array, columns = encoder_feature_names)
Data = pd.concat([new_df.reset_index(drop=True), encoder_vars_df.reset_index(drop=True)], axis = 1)
Data.drop(categorical_vars, axis = 1, inplace = True)

In [20]:
Data.head()

,movieId,title,userId_x,tag,timestamp_x,userId_y,rating,timestamp_y,genres_Action,genres_Action|Adventure,...,genres_Romance|Thriller,genres_Romance|War,genres_Romance|Western,genres_Sci-Fi,genres_Sci-Fi|IMAX,genres_Sci-Fi|Thriller,genres_Sci-Fi|Thriller|IMAX,genres_Thriller,genres_War,genres_Western
0,1,Toy Story (1995),336.0,pixar,1970-01-01 00:00:01.139045764,1,4.0,1970-01-01 00:00:00.964982703,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),62.0,fantasy,1970-01-01 00:00:01.528843929,6,4.0,1970-01-01 00:00:00.845553522,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),289.0,moldy,1970-01-01 00:00:01.143424860,1,4.0,1970-01-01 00:00:00.964981247,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),NaN,NaN,NaT,6,3.0,1970-01-01 00:00:00.845554349,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),474.0,pregnancy,1970-01-01 00:00:01.137373903,6,5.0,1970-01-01 00:00:00.845553938,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
